In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import json
import pickle
from replay_buffer.ber import BlockReplayBuffer
from ray.rllib.utils.replay_buffers import ReplayBuffer
from replay_buffer.mpber import MultiAgentPrioritizedBlockReplayBuffer
from ray.rllib.utils.replay_buffers.multi_agent_prioritized_replay_buffer import MultiAgentPrioritizedReplayBuffer

/home/seventheli/conda/envs/hpc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_configs(base_path):
    configs = {}
    for env in os.listdir(base_path):
        env_path = os.path.join(base_path, env)
        if os.path.isdir(env_path):
            configs[env] = {}
            for experiment in os.listdir(env_path):
                experiment_path = os.path.join(env_path, experiment)
                if os.path.isdir(experiment_path):
                    config_file = os.path.join(experiment_path, experiment + "_config.pyl")
                    if os.path.exists(config_file):
                        with open(config_file, "rb") as f:
                            configs[env][experiment] = pickle.load(f)
                    else:
                        print("config not existing for %s in %s" % (experiment, env))
    return configs

base_path = "/home/seventheli/JADE_checkpoints/"
configs = generate_configs(base_path)

config not existing for DQN_ER_SpaceInvadersNoFrameskip-v4_511703 in SpaceInvaders


In [3]:
data_path = "./jade_apex.csv"

In [4]:
mapper = {
    MultiAgentPrioritizedReplayBuffer: "DPER",
    MultiAgentPrioritizedBlockReplayBuffer: "DPBER",
}

In [5]:
data = pd.read_csv(data_path)
data['env'] = data['exp'].str.split('_').apply(lambda x: x[2].replace("NoFrameskip-v4", ""))
data["buffer"] = data.apply(lambda x: configs[x["env"]][x["exp"]].get("replay_buffer_config")["type"], axis=1)
data["buffer"] = data["buffer"].apply(lambda x: mapper[x])
data["size"] = data.apply(lambda x: configs[x["env"]][x["exp"]].get("replay_buffer_config").get("sub_buffer_size", 1), axis=1)
data.head()
data.to_csv("./jade_apex.csv", index=False)

In [6]:
max_exp = data.groupby(['env', 'buffer', 'size'])['exp'].apply(lambda x: x.value_counts().idxmax())

filtered_data = data[data['exp'].isin(max_exp.values)].reset_index(drop=True)
# Group by 'env', 'size', 'buffer' and aggregate the required metrics
summary = filtered_data.groupby(['env', 'size', 'buffer']).agg({
    'episode_reward_mean': 'last',  # Last episode reward mean
    'num_env_steps_sampled': 'sum', # Total number of data points
    'time_total_s': 'last',         # Total time
    'exp': 'unique'                 # Experiment names
}).reset_index()

# Add a column to indicate the number of unique experiments for each group
summary['num_experiments'] = summary['exp'].apply(len)

# Verify if there's only 1 experiment for each group
summary['single_experiment'] = summary['num_experiments'] == 1

# Display the summary
summary.single_experiment.value_counts()

# remove too short
filtered_data = filtered_data[filtered_data["exp"].isin([i[0] for i in summary[summary["time_total_s"]> 100000]["exp"]])]

In [7]:
filtered_data.groupby(['env', "buffer"])["time_total_s"].max().reset_index()

,env,buffer,time_total_s
0,Alien,DPBER,204351.006422
1,Amidar,DPBER,204105.807429
2,Assault,DPBER,203879.135070
3,Asterix,DPBER,203833.018097
4,Asteroids,DPBER,198468.946626
5,Atlantis,DPBER,197720.897195
6,BankHeist,DPBER,197800.318388
7,BattleZone,DPBER,196290.366011
8,BeamRider,DPBER,360056.846980
9,BeamRider,DPER,310839.486917


In [8]:
# Correctly compute the baseline difference for each environment
baseline_diff_corrected = filtered_data.groupby('env').apply(
    lambda x: x['episode_reward_mean'].max() - x['episode_reward_mean'].min()
)

# Using the corrected baseline difference to detect anomalies

# Function to detect anomalies based on the corrected baseline
def detect_anomalies_baseline(exp_data, env_baseline_diff):
    exp_diff = abs(exp_data['episode_reward_mean'].max() - exp_data['episode_reward_mean'].min())
    return exp_diff < abs(env_baseline_diff * 0.20)

anomalies_baseline_corrected = []

# Check each experiment in the data
for env, env_data in filtered_data.groupby('env'):
    env_baseline_diff = baseline_diff_corrected[env]
    for exp, exp_data in env_data.groupby('exp'):
        if detect_anomalies_baseline(exp_data, env_baseline_diff):
            anomalies_baseline_corrected.append(exp)

In [9]:
filtered_data = filtered_data[~filtered_data['exp'].isin(anomalies_baseline_corrected)].reset_index(drop=True)

In [10]:
filtered_data.to_csv("jade_apex_filtered.csv", index=False)

In [11]:
filtered_data

,episode_reward_max,episode_reward_min,episode_reward_mean,episodes_this_iter,num_env_steps_sampled,num_env_steps_trained,num_agent_steps_sampled,num_agent_steps_trained,num_weight_syncs,num_target_updates,...,learner_overall_throughput,target_net_update_time_ms,episodes_total,training_iteration,time_this_iter_s,time_total_s,size,exp,env,buffer
0,100.0,100.0,100.000000,8,26880,0,26880,0,168,NaN,...,NaN,NaN,8,1,31.200541,31.200541,16,APEX_DDQN_AlienNoFrameskip-v4_DPBER_518437,Alien,DPBER
1,100.0,100.0,100.000000,16,53280,34816,53280,34816,325,NaN,...,9568.821,NaN,24,2,30.138267,61.338807,16,APEX_DDQN_AlienNoFrameskip-v4_DPBER_518437,Alien,DPBER
2,990.0,100.0,176.562500,8,78400,436224,78400,436224,481,8.0,...,11606.370,7.245,32,3,40.167542,101.506349,16,APEX_DDQN_AlienNoFrameskip-v4_DPBER_518437,Alien,DPBER
3,990.0,100.0,201.666667,10,103680,841728,103680,841728,635,16.0,...,11729.788,10.832,42,4,40.386119,141.892469,16,APEX_DDQN_AlienNoFrameskip-v4_DPBER_518437,Alien,DPBER
4,1020.0,100.0,248.461538,10,128960,1250816,128960,1250816,792,24.0,...,9732.450,6.973,52,5,40.425197,182.317666,16,APEX_DDQN_AlienNoFrameskip-v4_DPBER_518437,Alien,DPBER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267341,2150.0,515.0,1315.100000,6,127910720,839138304,127910720,839138304,781035,16723.0,...,4803.690,10.027,29278,5071,33.193845,359898.757336,8,APEX_DDQN_SpaceInvadersNoFrameskip-v4_DPBER_51...,SpaceInvaders,DPBER
267342,2150.0,515.0,1312.300000,3,127935840,839302144,127935840,839302144,781189,16727.0,...,2319.787,11.075,29281,5072,33.767208,359932.524544,8,APEX_DDQN_SpaceInvadersNoFrameskip-v4_DPBER_51...,SpaceInvaders,DPBER
267343,2260.0,515.0,1298.600000,4,127960960,839467520,127960960,839467520,781343,16730.0,...,6187.669,5.276,29285,5073,32.214917,359964.739461,8,APEX_DDQN_SpaceInvadersNoFrameskip-v4_DPBER_51...,SpaceInvaders,DPBER
267344,2260.0,365.0,1306.400000,6,127986080,839630848,127986080,839630848,781497,16733.0,...,3916.615,8.996,29291,5074,34.269591,359999.009052,8,APEX_DDQN_SpaceInvadersNoFrameskip-v4_DPBER_51...,SpaceInvaders,DPBER
